In [2]:
from nltk.corpus import stopwords
import pandas as pd
from nltk.stem.wordnet import WordNetLemmatizer
import string

In [44]:
DATA = '../../Data/reviews.csv'
df = pd.read_csv(DATA)
count = 0
docs = []
restaurant_review_sentence_map = {}
for index, row in df.iterrows():
    rest_id = row['id']
    text = row['text']
    rating = row['stars']
    
    docs.append(text)
    
    if rest_id not in restaurant_review_sentence_map.keys():
        restaurant_review_sentence_map[rest_id] = {}
        for star in range(1,6):
            restaurant_review_sentence_map[rest_id][star] = []
    
    sentences = text.split('.')
    for sentence in sentences:
        restaurant_review_sentence_map[rest_id][rating].append(sentence)

    count += 1
print(count)

96543


In [4]:
stop = set(stopwords.words('english'))
exclude = set(string.punctuation) 
lemma = WordNetLemmatizer()

In [5]:
def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized

docs_clean = [clean(doc).split() for doc in docs]

In [6]:
import gensim
from gensim import corpora

# Creating the term dictionary of our courpus, where every unique term is assigned an index. dictionary = corpora.Dictionary(doc_clean)
dictionary = corpora.Dictionary(docs_clean)

# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
doc_term_matrix = [dictionary.doc2bow(doc) for doc in docs_clean]

In [7]:
import time
start = time.time()
# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

# Running and Trainign LDA model on the document term matrix.
ldamodel = Lda(doc_term_matrix, num_topics=10, id2word = dictionary, passes=1)
end = time.time()
print(end - start)

268.1303925514221


In [31]:
for i in range (20):
    print(ldamodel[dictionary.doc2bow(docs[i].split())])

[(0, 0.27872723267331601), (6, 0.45791956307513104), (7, 0.23534549388386319)]
[(2, 0.20820803553234662), (4, 0.10358936846660473), (7, 0.65776106163431147)]
[(2, 0.29675785247468622), (4, 0.27749362811031558), (6, 0.25094536476488249), (7, 0.14507333106546533), (8, 0.022685913858550448)]
[(0, 0.12149424704566701), (1, 0.040884946721853943), (2, 0.09086258775850406), (4, 0.51448564006653297), (5, 0.036250632989081974), (7, 0.18194028808835819), (8, 0.012477060005858428)]
[(2, 0.2423334451101089), (6, 0.54076073176602413), (7, 0.18005394893081902)]
[(4, 0.52672162470142858), (5, 0.15815975841795823), (7, 0.13229666267807205), (9, 0.17687984752750877)]
[(1, 0.23452475792905753), (4, 0.28350050196942644), (5, 0.17931578350006624), (6, 0.28932137590003987)]
[(1, 0.044976483324842345), (4, 0.29062138055430226), (5, 0.19476112761466161), (6, 0.16225388434489663), (7, 0.040945090969308616), (9, 0.2635216780951638)]
[(0, 0.50050165323505058), (4, 0.24321224127966015), (6, 0.21943432173907781)]

In [49]:
#must be manual
topic_aspect_map = {}
for topic in range(10):
    if topic < 3:
        topic_aspect_map[topic] = 'SERVICE'
    elif topic < 6:
        topic_aspect_map[topic] = 'FOOD'
    elif topic < 9:
        topic_aspect_map[topic] = 'COST'
    else:
        topic_aspect_map[topic] = 'AMBIENCE'

aspects = ['SERVICE', 'FOOD', 'COST', 'AMBIENCE']
threshold = 0.35

In [54]:
start = time.time()
res_map = {}
count = 1
for rest_id, rating_map in restaurant_review_sentence_map.items():
    print(rest_id, count)
    count += 1
    res_map[rest_id] = {}
    for aspect in aspects:
        res_map[rest_id][aspect] = {}
        for rating in range(1,6):
            res_map[rest_id][aspect][rating] = []
    for rating, sentences in rating_map.items():
        for sentence in sentences:
            sentence_topics = ldamodel[dictionary.doc2bow(sentence.split())]
            for topic, prob in sentence_topics:
                if prob > threshold:
                    res_map[rest_id][topic_aspect_map[topic]][rating].append(sentence)

end = time.time()
print(end - start)

LLDGTT7FExklkQvHebEjRg 1
mPDE4dMqAyLZNCR-tEYQ5w 2
IVc23uY-36WUNYoIbz42Fg 3
i3ogCoQD4MY_sR73GCk-BA 4
OiCxoAIoBvEWxf7p1qVVmA 5
aGbjLWzcrnEx2ZmMCFm3EA 6
guLkdetJ603Bw_A6tuecIg 7
FV16IeXJp2W6pnghTz2FAw 8
eltTG6JOr9cg5FRAgGy8Jw 9
szw8OGJlsqaA3i2oe7dn9A 10
_xmJB767rROtAy6XepMlkQ 11
mxrXVZWc6PWk81gvOVNOUw 12
ZgWUM594Yqk52IvBgoIrdA 13
Ht8mXLuqJSTPrU9kvzosUA 14
zTCCbg7mGslxACL5KlAPIQ 15
GhQQsSApIRfUmIcqzmSaGA 16
xxOQ7h6s3InV8o8DOVHlMA 17
l2lpBXx8jKl87J2szyJRuQ 18
-sC66z4SO3tR7nFCjfQwuQ 19
HyfFenprdpIA4rmKu6DW3g 20
b5cEoKR8iQliq-yT2_O0LQ 21
qT4w8n27RBQ_slz_FiK0CQ 22
AkOruz5CrCxUmXe1p_WoRg 23
v1aLzjTphPc6IGIy9k96vg 24
YrIfgzLj0DPStJ1ESAS_Qg 25
4UVhuOLaMm2-34SrW8y-ag 26
DqY2VlbRuQ3Mu2mlxd4xUQ 27
ryvMJK6AlbU35HKrlFT61w 28
uYVdp5LfCbX8S8ytnhdk-g 29
ntN85eu27C04nwyPa8IHtw 30
D_BMUHbZk67hzRcK3xQQmg 31
jf67Z1pnwElRSXllpQHiJg 32
jh7S78tc9oXCDjAjP_OrLw 33
_45p8kOA__aNycVVe-n-KA 34
uiadM74nDbEPGeGQFghK0g 35
VVeogjZya58oiTxK7qUjAQ 36
FURgKkRFtMK5yKbjYZVVwA 37
FV0BkoGOd3Yu_eJnXY15ZA 38
5paMI5Hhciyl09yGpzdoJ